In [ ]:
import pandas as pd
import numpy as np
import xlsxwriter

# Notes on Pre-Cleaning
Before this data was pulled into Python, column labels were changed from the original Qualtrics export. Column names were standardized to camel case and shortened to useable lengths.

Additionally, the data was run through the TAMU geocoding service (http://geoservices.tamu.edu/Services/Geocode/) to get latitude adn longitude for each address.

In [ ]:
#read in the simplified final stakeholders - these are our nodes
df = pd.read_csv('FinalStakeholders-simplified.csv')

df.info()

# Kumu Node Prep
Kumu requires multi-value fields to be pipe-delimited. First, clean for Kumu. We'll make different assumptions for Kumu than we will for SNA analysis.

In [ ]:
#these are all the columns we won't show on Kumu
dropColumns = ['ERGMFlag', 'Email', 'Address', 'City', 'State', 'Zip', 'HasSecondJob', 'IncomePercent', 'SocialMediaFlag', 'SocialMediaChoices', 'HoursOnSocialMedia','AnswerSources','AnswerSourcesOther','WebsiteFlag','NetworkOverlap', 'IntrovertExtrovertScale', 'PersonalSelfDependence', 'WorkSelfDependence','CollaborativeEconomicAdvantage', 'CollaborationsNotWorthIt', 'SocialMediaOther', 'OnlineMarketingFlag','Latitude', 'Longitude']
#make a copy, drop unused columns and fill in blanks
kumu = (df.copy()
        .drop(dropColumns, axis=1)
        .fillna({'Location': '',
               'Roles': '', 'RolesOther': '', 'BusinessName': '',
                'Employer': '',  'VendorMarkets': '', 'ConsumerMarkets': '',
               'GroupAffiliations': '', 'GroupAffiliationsOther': '', 
                 'Website': ''}))

#all of these columns 
listCols = ['Roles', 'VendorMarkets', 'ConsumerMarkets', 'GroupAffiliations']

#clean up this: (Growing Connections, Hulings Rice Food Center)
kumu['GroupAffiliations'] = kumu['GroupAffiliations'].apply(lambda x: x.replace('(Growing Connections, Hulings Rice Food Center)', '(Growing Connections or Hulings Rice Food Center)'))

for col in listCols:
    kumu[col] = kumu[col].apply(lambda x: x.replace(',', '|'))   
    
 
kumu.head()

# Kumu Edge Prep
For Kumu, we want the words (not the numbers) for our edge weights. We also want to drop the sensitive data. First we'll read in the file, drop unneeded columns and clean up the headers.

In [ ]:
kumuEdges = pd.read_csv('FromQualtricsEdges.csv', header=1)
kumuEdges = (kumuEdges.drop([0], axis=0)
         .drop(['Start Date', 'End Date', 'Response Type', 'IP Address', 'Progress', 
                'Duration (in seconds)', 'Finished', 'Recorded Date', 'Response ID', 'Recipient Last Name',
               'Recipient First Name', 'Recipient Email', 'Location Latitude', 'Location Longitude',
               'Distribution Channel', 'User Language'], axis=1))

#update the Philomena row 
kumuEdges['External Data Reference'] = kumuEdges['External Data Reference'].apply(lambda x: 'Philomena Kebec (GLIFWC)' if x == 'Philomena Kebec (GIFWC)' else x )

colNames = list(kumuEdges.columns)
newColNames = {'External Data Reference': 'Label'}
for c in colNames:
    if "Choose" in c:
        newColNames[c] = str.split(c, ' - ')[1]
    
#rename the columns and set the index
kumuEdges = kumuEdges.rename(columns=newColNames).set_index('Label').fillna('Unknown')

#fix the Philomena column
kumuEdges['Philomena Kebec (GLIFWC)'] = kumuEdges['Philomena Kebec (GIFWC)']
kumuEdges.drop(['Philomena Kebec (GIFWC)'], inplace=True, axis=1)




kumuEdges.head(10)

In [ ]:
#get the cleaned up column names
colNames = list(kumuEdges.columns)
#set up 3 lists for holding our 3 columns of data
kFrom = []
kTo = []
kType = []
kWeights = []

weights = {
    'Co-exist': 1,
    'Communicate': 2,
    'Coordinate': 3,
    'Collaborate': 4
}

#iterate through each row and column and get the edge combinations and their type
for i,j in kumuEdges.iterrows():
    #for each column
    for c in colNames:
        connection = ''.join(kumuEdges.loc[[i],[c]].values[0])
        if connection != 'Unknown':
            #don't include self-loops, if anyone set one
            if(i != c):
                kFrom.append(i)
                kTo.append(c)
                kType.append(connection)
                kWeights.append(weights[connection])

dfKEdgeList = pd.DataFrame({'From': kFrom, 'To':kTo, 'Type': kType, 'Weight': kWeights})
dfKEdgeList


# Write to Excel
For Kumu, we want one excel document with 2 sheets

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('kumuData.xlsx', engine='xlsxwriter', options={'index': False})

# Write each dataframe to a different worksheet.
#nodes
kumu.to_excel(writer, sheet_name='Elements', index=False)
#edges
dfKEdgeList.to_excel(writer, sheet_name='Connections', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()